In [1]:
import torch
from torch import nn
from d2l import limutorch as d2l

"""互相关运算"""
def corr2d(X, K):
  """计算二维相关运算"""
  h, w = K.shape
  Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
  for i in range(Y.shape[0]):
    for j in range(Y.shape[1]):
      Y[i, j] = (X[i:i+h, j:j+w] * K).sum()
  return Y

In [4]:
"""验证上述二维互相关运算的输出"""
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

In [ ]:
"""实现二维卷积层"""
class Conv2D(nn.Module):
  def __init__(self, kernel_size):
    super().__init__()
    self.weight = nn.Parameter(torch.rand(kernel_size))
    self.bias = nn.Parameter(torch.zeros(1))

  def forward(self, x):
    return corr2d(x, self.weight) + self.bias

In [5]:
"""卷积层的一个简单应用：检测图像中不同颜色的边缘"""
X = torch.ones((6,8))
X [:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [8]:
K = torch.tensor([[1.0, -1.0]])

In [9]:
"""输出Y中1代表从白色到黑色边缘，-1代表从黑色到白色边缘"""
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [10]:
"""[1.0, -1.0]卷积核只能检测垂直边缘，不能检测水平边缘"""
corr2d(X.t(), K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [11]:
"""学习由X生成Y的卷积核"""
conv2d = nn.Conv2d(1, 1, kernel_size=(1, 2), bias=False)

X = X.reshape((1, 1, 6, 8)) ## [x, y, m ,n] x是批量大小， y是通道数， [m, n]表示输入元素尺寸
Y = Y.reshape((1, 1, 6, 7))

for i in range(10):
  Y_hat = conv2d(X)
  l = (Y_hat - Y) ** 2
  conv2d.zero_grad()
  l.sum().backward()
  conv2d.weight.data[:] -= 3e-2 * conv2d.weight.grad
  if (i + 1)% 2 == 0:
    print(f'batch{i+1}, loss{l.sum():.3f}')

batch2, loss10.878
batch4, loss2.446
batch6, loss0.664
batch8, loss0.216
batch10, loss0.079


In [14]:
"""所学卷积核的权重张量"""
conv2d.weight.data, conv2d.weight.data.reshape((1,2))

(tensor([[[[ 1.0128, -0.9573]]]]), tensor([[ 1.0128, -0.9573]]))